### Finance Jobs & Resumes

##### Items:
* job listings:
    * Indeed
* resumes:
    * indeed
    * zip recruiter
    * angel list
    

##### Plan:
* job listings:
    * Indeed: api
* resumes:
    * indeed: requests to indeed resume search page, store all resume links
    * zip recruiter: selenium to login and then find all resume pdf links
    * angel list: use mahyar's login to and then same as zip recruiter
* html template for listings
* create two df objects to store all scraped data:
    * job listings:
        * schema:
            * generated id
            * listing url
            * source
            * job title
            * local path and pdf file name
    * resumes:
        * schema:
            * generated id
            * url
            * source
            * resume name
            * local path and pdf file name
        * notes:
            * for indeed, first scrape all pdf links and then download
* 10,000+ for each  


##### Nice to Haves:
* html template for the resumes

In [1]:
import requests
from lxml import html
from bs4 import BeautifulSoup
import json

## Job Listings

### Indeed

In [ ]:
publisher = 5219489624660681

In [ ]:
### Set API search parameters.
params = {'publisher': publisher,
            'format': 'json', # json or xml
            'l' : 'New+York', # city
            'useragent': 'Mozilla',
            'sort' : 'date', # sort by: relevance or date
            'q': '', #search query
            'v' : 2,
            'limit': 100
            }

In [ ]:
job_titles = ['[Financial Operations]', 'Financial Planning Analysis', 'Revenue Manager',\
              'CFO', 'Finance Manager', 'Financial Director']

In [ ]:
def get_indeed_jobs(params):
    url = "http://api.indeed.com/ads/apisearch?"
    results = []
    for job_title in job_titles:
        params['q'] = job_title
        request = url
        for k, v in params.items():
            request += '&{0}={1}'.format(k,v)
        r = requests.get(request)
        results.append(r.json()['results'])

    return results

In [ ]:
#get_indeed_jobs(params)

To Do:
 * create a function to check if the job title is relevant
 * for each listing url, use beatiful soup or lxml to extract relevant html content

## Resumes

### Indeed

To Do:
* function to search resumes by keyword
* function for finding pdf link for each resume
* function to iterate through each page of results

#### Find Resumes by Search Url

In [4]:
# function to find resumes by keyword
from http_request_randomizer.requests.proxy.requestProxy import RequestProxy

# Extract resumes from the search results
def find_resumes(resumelist):
    b = BeautifulSoup(resumelist.content, 'html.parser')
    columns = b.findAll('a', text="", attrs={'class' : 'app_link'})
    ans = {}
    # For every tag that contains a resume link
    for i, c in enumerate(columns):
        
        # Get proxy
        proxy = RequestProxy()
        
        # The 8th index holds the resume url
        resume_url = str(c).split('"')[7]
        resume = proxy.generate_proxied_request("https://www.indeed.com"+resume_url)
        resume = BeautifulSoup(resume.content, 'html.parser')
        
        # Add it to our dictionary
        ans[resume_url] = resume

    return ans

#### Create all search urls and pass them to the find_resumes function

In [5]:
# Iterate through job_titles_list and count increments at 50 per page
urls= {}
n = 150 #number of entries to read in from indeed.com
queries = []
job_titles_list = ['Financial+Operations', 'Financial+Planning+Analysis', 'Revenue+Manager',\
              'CFO', 'Finance+Manager', 'Financial+Director']

for job in job_titles_list:
    count, flag = [0, True]
    print("\n" + job)
    
    #While we have looked at fewer than n resumes
    while(count < n):
        # Get proxy
        req_proxy = RequestProxy()
        
        # Generate request
        queries.append("https://www.indeed.com/resumes?q="+job+"&l=New+York%2C+NY&cb=jt&start="+str(count))
        resumelist = req_proxy.generate_proxied_request(queries[-1])
        
        # Find resumes from the search query return
        ans = find_resumes(resumelist)
        
        # Print how many we found
        print("Resumes: +", len(ans))
        
        # Update our main url storage datastructure
        urls.update(ans)
        
        """
        # When we test it fully we will need try and except blocks but it slows us down until then
        except:
            print("Couldn't find resume: ", count)
            flag = False
        """
        # 50 results per page
        count+=50
        
    print("Count", count)

print("Total Urls:", len(urls))

2018-03-11 17:37:35,282 root   DEBUG    === Initialized Proxy Parsers ===
2018-03-11 17:37:35,284 root   DEBUG    	 FreeProxy Parser of 'http://free-proxy-list.net' with required bandwidth: '150' KBs
2018-03-11 17:37:35,285 root   DEBUG    	 ProxyForEU Parser of 'http://proxyfor.eu/geo.php' with required bandwidth: '1.0' KBs
2018-03-11 17:37:35,286 root   DEBUG    	 RebroWeebly Parser of 'http://rebro.weebly.com' with required bandwidth: '150' KBs
2018-03-11 17:37:35,288 root   DEBUG    	 SemairProxy Parser of 'https://premproxy.com/list/' with required bandwidth: '150' KBs
2018-03-11 17:37:35,290 root   DEBUG    =================================



Financial+Operations


2018-03-11 17:37:35,919 http_request_randomizer.requests.parsers.FreeProxyParser DEBUG    Proxy Invalid: <zip object at 0x104914e48>
2018-03-11 17:37:36,239 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy with low bandwidth: 0.178
2018-03-11 17:37:36,239 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy Invalid: <zip object at 0x104f40848>
2018-03-11 17:37:36,241 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy with low bandwidth: 0.435
2018-03-11 17:37:36,242 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy Invalid: <zip object at 0x104f40d48>
2018-03-11 17:37:36,243 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy with low bandwidth: 0.361
2018-03-11 17:37:36,244 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy Invalid: <zip object at 0x104f47108>
2018-03-11 17:37:36,460 http_request_randomizer.requests.parsers.RebroWeeblyParser WARNING  Prox

RR Status 200
Resumes: + 0


2018-03-11 17:37:38,554 http_request_randomizer.requests.parsers.FreeProxyParser DEBUG    Proxy Invalid: <zip object at 0x105ea3988>
2018-03-11 17:37:38,834 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy with low bandwidth: 0.178
2018-03-11 17:37:38,835 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy Invalid: <zip object at 0x10226e648>
2018-03-11 17:37:38,837 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy with low bandwidth: 0.435
2018-03-11 17:37:38,839 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy Invalid: <zip object at 0x1035c9748>
2018-03-11 17:37:38,841 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy with low bandwidth: 0.361
2018-03-11 17:37:38,842 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy Invalid: <zip object at 0x1035c92c8>
2018-03-11 17:37:39,064 http_request_randomizer.requests.parsers.RebroWeeblyParser WARNING  Prox

...

RR Status 200
Resumes: + 0


2018-03-11 17:40:08,342 http_request_randomizer.requests.parsers.FreeProxyParser DEBUG    Proxy Invalid: <zip object at 0x10629bb08>
2018-03-11 17:40:08,635 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy with low bandwidth: 0.178
2018-03-11 17:40:08,636 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy Invalid: <zip object at 0x106c96488>
2018-03-11 17:40:08,638 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy with low bandwidth: 0.435
2018-03-11 17:40:08,640 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy Invalid: <zip object at 0x106c96988>
2018-03-11 17:40:08,642 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy with low bandwidth: 0.361
2018-03-11 17:40:08,643 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy Invalid: <zip object at 0x106c96d08>
2018-03-11 17:40:09,031 http_request_randomizer.requests.parsers.RebroWeeblyParser WARNING  Prox

RR Status 200
Resumes: + 0


2018-03-11 17:40:10,561 http_request_randomizer.requests.parsers.FreeProxyParser DEBUG    Proxy Invalid: <zip object at 0x104a2f608>
2018-03-11 17:40:10,868 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy with low bandwidth: 0.178
2018-03-11 17:40:10,869 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy Invalid: <zip object at 0x1067c9988>
2018-03-11 17:40:10,870 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy with low bandwidth: 0.435
2018-03-11 17:40:10,871 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy Invalid: <zip object at 0x1069bba08>
2018-03-11 17:40:10,872 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy with low bandwidth: 0.361
2018-03-11 17:40:10,873 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy Invalid: <zip object at 0x1069bba88>
2018-03-11 17:40:11,111 http_request_randomizer.requests.parsers.RebroWeeblyParser WARNING  Prox

RR Status 200
Resumes: + 0
Count 150
Total Urls: 50


## TEST/DEBUG ABOVE CODE

In [10]:
print("Links checked:")
for i, q in enumerate(queries):    
    print(str(i+1)+')',q)

print("Found: ", len(urls), "resumes")
# Check 1 Resume url - doesn't work, the link opens differently than expected
url_keys = list(urls.keys())
print("\nSample Resume:", urls[url_keys[0]])

Links checked:
1) https://www.indeed.com/resumes?q=Financial+Operations&l=New+York%2C+NY&cb=jt&start=0
2) https://www.indeed.com/resumes?q=Financial+Operations&l=New+York%2C+NY&cb=jt&start=50
3) https://www.indeed.com/resumes?q=Financial+Operations&l=New+York%2C+NY&cb=jt&start=100
4) https://www.indeed.com/resumes?q=Financial+Planning+Analysis&l=New+York%2C+NY&cb=jt&start=0
5) https://www.indeed.com/resumes?q=Financial+Planning+Analysis&l=New+York%2C+NY&cb=jt&start=50
6) https://www.indeed.com/resumes?q=Financial+Planning+Analysis&l=New+York%2C+NY&cb=jt&start=100
7) https://www.indeed.com/resumes?q=Revenue+Manager&l=New+York%2C+NY&cb=jt&start=0
8) https://www.indeed.com/resumes?q=Revenue+Manager&l=New+York%2C+NY&cb=jt&start=50
9) https://www.indeed.com/resumes?q=Revenue+Manager&l=New+York%2C+NY&cb=jt&start=100
10) https://www.indeed.com/resumes?q=CFO&l=New+York%2C+NY&cb=jt&start=0
11) https://www.indeed.com/resumes?q=CFO&l=New+York%2C+NY&cb=jt&start=50
12) https://www.indeed.com/resume

In [7]:
)# function for finding pdf link for each resume

SyntaxError: invalid syntax (<ipython-input-7-44e11ceac1d8>, line 1)

### Zip Recruiter

To Do:
* login to zip recruiter 
* search resumes by keyword
* finding pdf link for each resume
* iterate through each page of results

### Angel List

To Do:
* login to zip recruiter 
* search resumes by keyword
* finding pdf link for each resume
* iterate through each page of results

### Overall
* function to save pdf links (wget)